In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (753).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (74).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (87).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (82).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (780).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (779).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (777).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (794).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (767).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (776).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (81).jpg
/kaggle/input/data-set/Data_set/Augmented Image/Downy Mildew/Downy Mildew (765).

In [2]:
# Set your own project id here
PROJECT_ID = 'https://drive.google.com/drive/u/0/my-drive'
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [3]:
# Cloud Storage
from google.cloud import storage
storage_client = storage.Client(project='')

In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


def trainning(data_dir, output_file):
	# Prepare training data
	data = [] 														# List of saved data
	labels = [] 													# List of stored corresponding labels

	# Loop through all folders in the root directory
	for class_name in os.listdir(data_dir):
	    class_dir = os.path.join(data_dir, class_name)
	    if os.path.isdir(class_dir): 								# Make sure the subfolder is a folder and not a file
	       	# Loop through all image files in subfolders
	        for image_name in os.listdir(class_dir):
	            if image_name.endswith(".jpg"):						# Only use images in .jpg format
	                image_path = os.path.join(class_dir, image_name)
	                img = cv2.imread(image_path)

	                # Resize the image to the desired size (In this case: 224x224)
	                img = cv2.resize(img, (224, 224))

	                # Normalize pixels to the range [0, 1]
	                img = img / 255.0

	                # Add photos and labels to the corresponding list
	                data.append(img)
	                labels.append(class_name)

	# Convert list of data and labels to NumPy array
	data = np.array(data)
	labels = np.array(labels)

	# Create a mapping from label strings to integers
	label_to_int = {label: index for index, label in enumerate(np.unique(labels))}

	# Convert label string to integer
	labels_as_int = [label_to_int[label] for label in labels]

	# Convert the label array to one-hot encoding
	one_hot_labels = to_categorical(labels_as_int)

	# Divide the data into training set and test set
	train_data, val_data, train_labels, val_labels = train_test_split(data, one_hot_labels, test_size=0.2, random_state=42)

	# VGG16-like model with 5 blocks
	model = Sequential([
	    layers.Input(shape=(224, 224, 3)),   # Input with 224x224 size and 3 color channels (RGB)

	    # Block 1
	    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
	    layers.MaxPooling2D((2, 2)),

	    # Block 2
	    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
	    layers.MaxPooling2D((2, 2)),

	    # Block 3
	    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
	    layers.MaxPooling2D((2, 2)),

	    # Block 4
	    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
	    layers.MaxPooling2D((2, 2)),

	    # Block 5
	    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
	    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
	    layers.MaxPooling2D((2, 2)),

	    # Flatten
	    layers.Flatten(),

	    # Fully connected layers
	    layers.Dense(4096, activation='relu'),
	    layers.Dense(4096, activation='relu'),
	    layers.Dense(8, activation='softmax')  # 8 output layers (number of diseases)
	])

	# Define ModelCheckpoint callback
	checkpoint_dir = '/content/drive/MyDrive/VGG16-main/models/checkpoints'  				# Directory to save checkpoints
	os.makedirs(checkpoint_dir, exist_ok=True)  		# Create the folder if it doesn't already exist
	checkpoint_path = os.path.join(checkpoint_dir, "model_weights.h5")

	checkpoint_callback = ModelCheckpoint(
	    filepath=checkpoint_path,
	    save_best_only=True,  							# Save only the best model based on monitor criteria (default is validation loss)
	    monitor='val_loss',  							# Check according to validation loss
	    verbose=1,  									# Show a notification when the model is saved
	)

	# Compile the model
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

	# Model training
	model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels), callbacks=[checkpoint_callback])

	# Evaluate the model
	model.evaluate(val_data, val_labels)

	# Export model
	model.save(output_file)

if __name__ == "__main__":
	data_dir = "/kaggle/input/data-set/Data_set/VGG16-main/data/"
	output_dir = "/kaggle/working/"
	os.makedirs(output_dir, exist_ok=True)  			# Create the folder if it doesn't already exist
	output_path = os.path.join(output_dir, "version1.keras")
	trainning(data_dir, output_path)


Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 4.1134 - accuracy: 0.0969 
Epoch 1: val_loss improved from inf to 2.06952, saving model to /content/drive/MyDrive/VGG16-main/models/checkpoints/model_weights.h5
9/9 [==============================] - 291s 32s/step - loss: 4.1134 - accuracy: 0.0969 - val_loss: 2.0695 - val_accuracy: 0.2000
Epoch 2/10
9/9 [==============================] - ETA: 0s - loss: 2.0696 - accuracy: 0.1628 
Epoch 2: val_loss improved from 2.06952 to 2.04640, saving model to /content/drive/MyDrive/VGG16-main/models/checkpoints/model_weights.h5
9/9 [==============================] - 305s 34s/step - loss: 2.0696 - accuracy: 0.1628 - val_loss: 2.0464 - val_accuracy: 0.2000
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 2.0616 - accuracy: 0.1628 
Epoch 3: val_loss improved from 2.04640 to 2.04528, saving model to /content/drive/MyDrive/VGG16-main/models/checkpoints/model_weights.h5
9/9 [==============================] - 294s 33s/step 